In [23]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8856), started 0:16:04 ago. (Use '!kill 8856' to kill it.)

In [13]:
import datetime, os
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import preprocessing
import pandas as pd
from PIL import Image
from keras.utils import load_img
from keras.utils import img_to_array
# I don't think we need this function
# from keras.utils import array_to_img

In [14]:
training_csv = pd.read_csv('fairface_label_train.csv')
training_csv = training_csv.iloc[1:25]
training_csv = training_csv.reset_index()
training_images = []
for index, row in training_csv.iterrows():
    image = Image.open(row['file']).convert("L") # load_img(row['file'], color_mode="grayscale", target_size=(32,32))
    data = np.asarray(image)
    training_images.append(data)
#     print(f"Loaded image: {type(image)}")
#     print(f"image #{index} file name: {row['file']}")
#     print(f"image #{index} format: {image.format}")
#     print(f"image #{index} size: {image.size}")
    image_array = np.asarray(image) # img_to_array(image) (using the keras methods gives float32 type instead of uint8, not sure which we need)
#     print(f"Numpy array: {type(image_array)}")
#     print(f"image array #{index} type: {image_array.dtype}")
#     print(f"image array #{index} shape: {image_array.shape}")
testing_csv = pd.read_csv('fairface_label_val.csv')
testing_csv = testing_csv.iloc[1:25]
testing_csv = testing_csv.reset_index()
testing_images = []
for index, row in testing_csv.iterrows():
    image = Image.open(row['file']).convert("L")
    data = np.asarray(image)
    testing_images.append(data)

In [15]:
# Min-Max Scale the training_images based on their min and max values (per image, not just scaled from 0 to 255)
#                                                at least according to the google chat  ^^^
temp = []
for i in training_images:
    mymaxes = []
    mymins = []
    for j in i:
        mymaxes.append(max(j))
        mymins.append(min(j))
    mymax = max(mymaxes)
    mymin = min(mymins)
    tempcol = []
    for j in i:
        temprow = []
        for k in j:
            temprow.append((k - mymin) / (mymax - mymin))
        tempcol.append(temprow)
    temp.append(tempcol)
training_images = temp
training_images = np.array(training_images)
# Scale the testing_images the same way
temp = []
for i in testing_images:
    mymaxes = []
    mymins = []
    for j in i:
        mymaxes.append(max(j))
        mymins.append(min(j))
    mymax = max(mymaxes)
    mymin = min(mymins)
    tempcol = []
    for j in i:
        temprow = []
        for k in j:
            temprow.append((k - mymin) / (mymax - mymin))
        tempcol.append(temprow)
    temp.append(tempcol)
testing_images = temp
testing_images = np.array(testing_images)

In [16]:
#also set up other vectors for training/validation
#Gender
genders = training_csv["gender"]
gens = []
for i in range(len(genders)):
    genset = np.zeros(2)
    if "Female" in genders[i]:
        genset[0] = 1
    else:
        genset[1] = 1
    gens.append(genset)
genders = gens.copy()

gendersVal = testing_csv["gender"]
gens = []
for i in range(len(gendersVal)):
    genset = np.zeros(2)
    if "Female" in gendersVal[i]:
        genset[0] = 1
    else:
        genset[1] = 1
    gens.append(genset)
gendersVal = gens.copy()

#Race
racs = []
races = training_csv["race"]
for i in races:
    if i not in racs:
        racs.append(i)
tempRaces = []
for i in range(len(races)):
    racset = np.zeros(len(racs))
    racset[racs.index(races[i])] = 1
    tempRaces.append(racset)
races = tempRaces.copy()

racesVal = testing_csv["race"]
tempRaces = []
for i in range(len(racesVal)):
    racset = np.zeros(len(racs))
    racset[racs.index(racesVal[i])] = 1
    tempRaces.append(racset)
racesVal = tempRaces.copy()
numRaces = len(racs)

genders = np.array(genders)
gendersVal = np.array(gendersVal)
races = np.array(races)
racesVal = np.array(racesVal)

## Task 1: Fully Connected Neural Network
##### 1. Build a feed forward neural network with the following specs (for 2 separate tasks)
###### a. Hidden layer 1: 1024 neurons with tanh activation function in each 
###### b. Hidden layer 2: 512 neurons with sigmoid activation function in each 
###### c. Hidden layer 3: 100 neurons with reLu
###### d. Output layer: n (depending on the task) representing n classes, using softmax activation

In [17]:
# fmodel = Sequential()
# fmodel.add(keras.Input(shape=(32,32,1)))
# fmodel.add(layers.Dense(1024, activation='tanh'))
# fmodel.add(layers.Dense(512, activation='sigmoid'))
# fmodel.add(layers.Dense(100, activation='relu'))
# # find n for the task, add this layer
# fmodel.add(layers.Dense(n, activation='softmax'))

### Task 1, Classification Task A: Gender

In [18]:
# classify gender
famodel = Sequential()
famodel.add(keras.Input(shape=(32,32,1)))
famodel.add(layers.Flatten())
famodel.add(layers.Dense(1024, activation='tanh'))
famodel.add(layers.Dense(512, activation='sigmoid'))
famodel.add(layers.Dense(100, activation='relu'))
n = 2
famodel.add(layers.Dense(n, activation='softmax'))
# min max already taken care of, use mini-batch gradient descent to optimize categorical cross entropy, 
# record and graph each epoch's epoch-accuracy and epoch-loss for both the training and validation set
# also graph the confusion matrix
opt = keras.optimizers.SGD(learning_rate=0.01)
famodel.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S_Task1Genders"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

famodel.fit(training_images, genders, epochs=100, batch_size=10, validation_data=(testing_images, gendersVal), callbacks=[tensorboard_callback])

Epoch 1/100
3/3 [==============================] - 1s 338ms/step - loss: 0.6651 - accuracy: 0.5833 - val_loss: 0.6768 - val_accuracy: 0.5833
Epoch 2/100
3/3 [==============================] - 0s 195ms/step - loss: 0.6600 - accuracy: 0.6250 - val_loss: 0.6694 - val_accuracy: 0.5833
Epoch 3/100
3/3 [==============================] - 0s 196ms/step - loss: 0.6828 - accuracy: 0.6250 - val_loss: 0.7578 - val_accuracy: 0.5417
Epoch 4/100
3/3 [==============================] - 0s 198ms/step - loss: 0.6845 - accuracy: 0.5417 - val_loss: 0.8582 - val_accuracy: 0.4583
Epoch 5/100
3/3 [==============================] - 0s 206ms/step - loss: 0.6658 - accuracy: 0.5000 - val_loss: 0.9745 - val_accuracy: 0.4583
Epoch 6/100
3/3 [==============================] - 0s 198ms/step - loss: 0.7929 - accuracy: 0.6250 - val_loss: 0.7507 - val_accuracy: 0.5417
Epoch 7/100
3/3 [==============================] - 0s 220ms/step - loss: 0.7052 - accuracy: 0.5000 - val_loss: 0.9035 - val_accuracy: 0.4583
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 1s 249ms/step - loss: 0.4217 - accuracy: 0.7083 - val_loss: 0.7457 - val_accuracy: 0.5833
Epoch 60/100
3/3 [==============================] - 0s 233ms/step - loss: 0.4280 - accuracy: 0.8750 - val_loss: 0.7506 - val_accuracy: 0.5833
Epoch 61/100
3/3 [==============================] - 0s 230ms/step - loss: 0.3843 - accuracy: 0.9167 - val_loss: 0.8535 - val_accuracy: 0.4583
Epoch 62/100
3/3 [==============================] - 0s 228ms/step - loss: 0.3842 - accuracy: 0.8333 - val_loss: 0.8670 - val_accuracy: 0.4583
Epoch 63/100
3/3 [==============================] - 0s 238ms/step - loss: 0.4131 - accuracy: 0.8750 - val_loss: 0.8577 - val_accuracy: 0.4583
Epoch 64/100
3/3 [==============================] - 1s 245ms/step - loss: 0.3991 - accuracy: 0.8333 - val_loss: 0.9465 - val_accuracy: 0.4583
Epoch 65/100
3/3 [==============================] - 1s 269ms/step - loss: 0.3977 - accuracy: 0.8333 - val_loss: 0.9620 - val_accuracy: 0.5000
Epoch 

### Task 1, Classification Task B: Race

In [19]:
# classify race
fbmodel = Sequential()
fbmodel.add(keras.Input(shape=(32,32,1)))
fbmodel.add(layers.Flatten())
fbmodel.add(layers.Dense(1024, activation='tanh'))
fbmodel.add(layers.Dense(512, activation='sigmoid'))
fbmodel.add(layers.Dense(100, activation='relu'))
# find n for the task, add this layer
n = numRaces
fbmodel.add(layers.Dense(n, activation='softmax'))
# min max already taken care of, use mini-batch gradient descent to optimize categorical cross entropy, 
# record and graph each epoch's epoch-accuracy and epoch-loss for both the training and validation set
# also graph the confusion matrix
opt = keras.optimizers.SGD(learning_rate=0.01)
fbmodel.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S_Task1Races"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

fbmodel.fit(training_images, races, epochs=100, batch_size=10, validation_data=(testing_images, racesVal), callbacks=[tensorboard_callback])
# if you have the setup and training in different blocks in the notebook, 
# you can accidentally train it multiple times, compounding how well it gets trained, 
# so keep the training in the same block as the setup

Epoch 1/100
3/3 [==============================] - 1s 333ms/step - loss: 1.9543 - accuracy: 0.2917 - val_loss: 2.3043 - val_accuracy: 0.0833
Epoch 2/100
3/3 [==============================] - 0s 240ms/step - loss: 1.8849 - accuracy: 0.2917 - val_loss: 2.2163 - val_accuracy: 0.0417
Epoch 3/100
3/3 [==============================] - 0s 220ms/step - loss: 1.8811 - accuracy: 0.2083 - val_loss: 2.1187 - val_accuracy: 0.0833
Epoch 4/100
3/3 [==============================] - 0s 219ms/step - loss: 1.8524 - accuracy: 0.1667 - val_loss: 2.0820 - val_accuracy: 0.1250
Epoch 5/100
3/3 [==============================] - 0s 224ms/step - loss: 1.8127 - accuracy: 0.2917 - val_loss: 2.0555 - val_accuracy: 0.2083
Epoch 6/100
3/3 [==============================] - 0s 197ms/step - loss: 1.8311 - accuracy: 0.2083 - val_loss: 2.0728 - val_accuracy: 0.1250
Epoch 7/100
3/3 [==============================] - 0s 238ms/step - loss: 1.8191 - accuracy: 0.2500 - val_loss: 2.1775 - val_accuracy: 0.0833
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 0s 199ms/step - loss: 1.4130 - accuracy: 0.5000 - val_loss: 2.3953 - val_accuracy: 0.0833
Epoch 60/100
3/3 [==============================] - 0s 195ms/step - loss: 1.4251 - accuracy: 0.3750 - val_loss: 2.3707 - val_accuracy: 0.0833
Epoch 61/100
3/3 [==============================] - 0s 191ms/step - loss: 1.4079 - accuracy: 0.4583 - val_loss: 2.3188 - val_accuracy: 0.0417
Epoch 62/100
3/3 [==============================] - 0s 194ms/step - loss: 1.4255 - accuracy: 0.6250 - val_loss: 2.4200 - val_accuracy: 0.0833
Epoch 63/100
3/3 [==============================] - 0s 189ms/step - loss: 1.4369 - accuracy: 0.3750 - val_loss: 2.4207 - val_accuracy: 0.0833
Epoch 64/100
3/3 [==============================] - 0s 204ms/step - loss: 1.3875 - accuracy: 0.4583 - val_loss: 2.3879 - val_accuracy: 0.0833
Epoch 65/100
3/3 [==============================] - 0s 183ms/step - loss: 1.4368 - accuracy: 0.3750 - val_loss: 2.4045 - val_accuracy: 0.1250
Epoch 

## Task 2: Convolutional Network
#### 2. Build a convolutional network (also for 2 tasks)
###### a. conv layer with 40 feature detectors, kernel size 5x5, reLu activation, stride=1, no padding
###### b. max pooling layer size 2x2
###### c. fully connected layer, 100 neurons reLu activation
###### d. output layer: n neurons (depending on task), using softmax activation

In [20]:
# cmodel = Sequential()
# cmodel.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation='relu', input_shape=(32,32,1), strides=1, padding='valid'))
# cmodel.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
# cmodel.add(layers.Dense(100, activation='relu'))
# #find n for the task
# cmodel.add(layers.Dense(n, activation='softmax'))

### Task 2, Classification Task A: Gender

In [21]:
# classify gender
camodel = Sequential()
camodel.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation='relu', input_shape=(32,32,1), strides=1, padding='valid'))
camodel.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
camodel.add(layers.Flatten())
camodel.add(layers.Dense(100, activation='relu'))
n = 2
camodel.add(layers.Dense(n, activation='softmax'))

opt = keras.optimizers.SGD(learning_rate=0.01)
camodel.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S_Task2Genders"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

camodel.fit(training_images, genders, epochs=100, batch_size=10, validation_data=(testing_images, gendersVal), callbacks=[tensorboard_callback])

Epoch 1/100
3/3 [==============================] - 1s 219ms/step - loss: 0.6809 - accuracy: 0.6250 - val_loss: 0.7473 - val_accuracy: 0.4583
Epoch 2/100
3/3 [==============================] - 0s 121ms/step - loss: 0.6484 - accuracy: 0.5833 - val_loss: 0.7278 - val_accuracy: 0.4583
Epoch 3/100
3/3 [==============================] - 0s 110ms/step - loss: 0.6351 - accuracy: 0.5833 - val_loss: 0.7350 - val_accuracy: 0.5000
Epoch 4/100
3/3 [==============================] - 0s 126ms/step - loss: 0.6260 - accuracy: 0.5417 - val_loss: 0.7150 - val_accuracy: 0.4583
Epoch 5/100
3/3 [==============================] - 0s 133ms/step - loss: 0.6198 - accuracy: 0.5833 - val_loss: 0.7073 - val_accuracy: 0.4583
Epoch 6/100
3/3 [==============================] - 0s 126ms/step - loss: 0.6101 - accuracy: 0.6667 - val_loss: 0.7139 - val_accuracy: 0.4583
Epoch 7/100
3/3 [==============================] - 0s 107ms/step - loss: 0.6042 - accuracy: 0.5833 - val_loss: 0.6663 - val_accuracy: 0.6250
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 0s 106ms/step - loss: 0.2891 - accuracy: 0.9583 - val_loss: 0.7648 - val_accuracy: 0.6667
Epoch 60/100
3/3 [==============================] - 0s 122ms/step - loss: 0.2789 - accuracy: 0.9167 - val_loss: 0.9765 - val_accuracy: 0.5000
Epoch 61/100
3/3 [==============================] - 0s 108ms/step - loss: 0.2709 - accuracy: 0.8750 - val_loss: 0.7835 - val_accuracy: 0.7083
Epoch 62/100
3/3 [==============================] - 0s 123ms/step - loss: 0.2777 - accuracy: 0.9583 - val_loss: 0.7870 - val_accuracy: 0.7083
Epoch 63/100
3/3 [==============================] - 0s 113ms/step - loss: 0.3044 - accuracy: 0.8750 - val_loss: 0.8640 - val_accuracy: 0.6250
Epoch 64/100
3/3 [==============================] - 0s 130ms/step - loss: 0.2492 - accuracy: 0.9583 - val_loss: 0.8566 - val_accuracy: 0.6250
Epoch 65/100
3/3 [==============================] - 0s 114ms/step - loss: 0.2541 - accuracy: 0.9167 - val_loss: 0.8114 - val_accuracy: 0.6250
Epoch 

### Task 2, Classification Task B: Race

In [22]:
# classify race
cbmodel = Sequential()
cbmodel.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation='relu', input_shape=(32,32,1), strides=1, padding='valid'))
cbmodel.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
cbmodel.add(layers.Flatten())
cbmodel.add(layers.Dense(100, activation='relu'))
n = numRaces
cbmodel.add(layers.Dense(n, activation='softmax'))

opt = keras.optimizers.SGD(learning_rate=0.01)
cbmodel.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S_Task2Races"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

cbmodel.fit(training_images, races, epochs=100, batch_size=10, validation_data=(testing_images, racesVal), callbacks=[tensorboard_callback])

Epoch 1/100
3/3 [==============================] - 1s 233ms/step - loss: 1.9265 - accuracy: 0.2083 - val_loss: 1.9763 - val_accuracy: 0.0833
Epoch 2/100
3/3 [==============================] - 0s 131ms/step - loss: 1.8689 - accuracy: 0.2917 - val_loss: 2.0022 - val_accuracy: 0.1250
Epoch 3/100
3/3 [==============================] - 0s 129ms/step - loss: 1.8399 - accuracy: 0.3750 - val_loss: 2.0322 - val_accuracy: 0.1250
Epoch 4/100
3/3 [==============================] - 0s 134ms/step - loss: 1.8002 - accuracy: 0.2917 - val_loss: 2.0688 - val_accuracy: 0.1250
Epoch 5/100
3/3 [==============================] - 0s 121ms/step - loss: 1.7718 - accuracy: 0.3333 - val_loss: 2.0863 - val_accuracy: 0.1250
Epoch 6/100
3/3 [==============================] - 0s 114ms/step - loss: 1.7737 - accuracy: 0.2500 - val_loss: 2.2298 - val_accuracy: 0.0833
Epoch 7/100
3/3 [==============================] - 0s 123ms/step - loss: 1.7718 - accuracy: 0.2917 - val_loss: 2.2050 - val_accuracy: 0.1250
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 0s 117ms/step - loss: 0.7809 - accuracy: 0.7500 - val_loss: 3.3404 - val_accuracy: 0.1250
Epoch 60/100
3/3 [==============================] - 0s 112ms/step - loss: 0.7035 - accuracy: 0.8750 - val_loss: 3.8907 - val_accuracy: 0.1250
Epoch 61/100
3/3 [==============================] - 0s 131ms/step - loss: 0.7963 - accuracy: 0.8333 - val_loss: 3.6609 - val_accuracy: 0.1250
Epoch 62/100
3/3 [==============================] - 0s 130ms/step - loss: 0.7317 - accuracy: 0.7083 - val_loss: 3.7261 - val_accuracy: 0.0833
Epoch 63/100
3/3 [==============================] - 0s 115ms/step - loss: 0.7322 - accuracy: 0.7917 - val_loss: 3.8137 - val_accuracy: 0.1250
Epoch 64/100
3/3 [==============================] - 0s 115ms/step - loss: 0.8327 - accuracy: 0.6250 - val_loss: 3.6346 - val_accuracy: 0.2083
Epoch 65/100
3/3 [==============================] - 0s 122ms/step - loss: 0.5974 - accuracy: 0.9167 - val_loss: 3.7990 - val_accuracy: 0.0833
Epoch 

## Task 3: Our own Convolutional Network
### Can we build our own network architecture to do better than either cmodel?

### Classification Task A: Gender

### Classification Task B: Race

## Task 4: Our own Convolutional Network to do both classifications simultaneously
### We can't use Sequential() for this one

## Task 5: Variational Auto-Encoder
### Use these specs (with some flexibility
###### a. at least 2 convolutional layers and 2 deconvolution layers
###### b. latent dimension should be at least 5
###### c. loss should be either MSE or binary cross entropy